<h3>Import Required Element</h3>
<p>Inorder to work properly, we need to import some library to help us.</p>

In [1]:
#-*- coding: utf-8 -*-
import urllib2
from urlparse import urlparse, parse_qs
from bs4 import BeautifulSoup
import sys
import time
import csv
from random import shuffle
import threading
import lxml

#Chinese encoding
import codecs

<h3>Use The Gloable Variables</h3>
<p>The variable we required</p>

In [2]:
#Set samlple to '' to crawl the whole data set at once
sample = ''
sleep = 0.1
links_file = 'App_Store_links.csv'

#Porgrame parameters
nav_site = "https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8"
alphabet = ['*','A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
            'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

<h3>Program Functions</h3>
<p>The major part of the program are running in this section</p>

In [ ]:
def site_open(site):
    URLError = ''
    try:
        #sets up request object.
        req = urllib2.Request(site)
        
        #adds the user-agnet info to request the object
        req.add_header("User-Agent","Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
        AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393")
        
        #opensite
        website = urllib2.urlopen(req)
        return website
    
    except URLError:
        print("Something occured that cause failure!" +
             "Please check this site" + site)
        pass
    
def soup_site(site):
    '''Opens site and turns it into a format to easily parse the DOM.'''
    return BeautifulSoup(site_open(site), "lxml")

<h3>Generate App links</h3>
<p>
    Generator function that outputs each genre in the app store when called to be used <br/>
    when getting general App Store Links.<br/>
    Yields a gener url.
</p>

In [ ]:
def generate_link_list(site):
    #open site and makes nabigatable DOM
    soup = soup_site(site)
    
    #creates array of genre links
    table = soup.find(id="genre-nav").find_all("a")
    
    for link in table:
        
        #outputs tuple with the link that will be written to a csv file
        yield link.get('href')
        
def app_link_list(site):
    '''
    Generate function that outputs a single specific app link
    in the app store when called to be used when getting App Store links.
    '''
    
    soup = soup_site(site)
    
    table = soup.find(id="selectedcontent").find_all("a")

    for link in table:
        #outputs tuple with the link that will be written to a csv file
        yield (link.get('href'),)
        
def pages_number(site):
    number = 0
    
    req = urllib2.Request(site)
    req.add_header("User-Agent","Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393")
      #opens up site
    website = urllib2.urlopen(req)    
    soup = BeautifulSoup(website, "html5lib")
    
    for test in soup.find(id = "main").find(id = "selectedgenre").find_all("a")[-1]:
        try:
            return int(test)
        except:
             for page_each in soup.find(id = "main").find(id = "selectedgenre").find_all("a")[-2]:
                return int(page_each) + 1
        
def page_array(page_number):
    page_array = []
    for i in range(int(page_number)):
        page_array.extend([i+1])
    return page_array

def general_app_store_crawl(filname, sleep_time = float):
    #creates and/or appedns of csv file
    f = open(filname, 'ab')
    f.write(codecs.BOM_UTF8)
    
    writer = csv.writer(f, dialect = 'excel', quoting=csv.QUOTE_NONNUMERIC)
    
    loop_count = 1
    #grab everything form the database
    for i, link in enumerate(generate_link_list(nav_site)):
        print("Scraping genre " + str(i) + ".")
        
        #loop throgh the alphabet to genreate relevant sites and write to csv.
        for letter in alphabet:
            
            #reconstructs url
            link_site = link + "&letter=" + letter
            
            try:
                for a in page_array(pages_number(link_site)):
                    new_site = link + "&letter=" + letter + "&page=" + str(a) + "#page"
                    print new_site
                    loop_count += 1
                    for app in app_link_list(new_site):
                        writer.writerow(app)

                while int(pages_number(new_site)) >= loop_count:
                
                    for k in range(loop_count, int(pages_number(new_site)) + 1):
                        new_site = link + "&letter=" + letter + "&page=" + str(k) + "#page"
                        print new_site
                        loop_count += 1
                        for app in app_link_list(new_site):
                            writer.writerow(app)
                
                loop_count = 1
            except:
                new_site = link + "&letter=" + letter
                print new_site
                for app in app_link_list(new_site):
                    writer.writerow(app)
                
            #pause for politeness
            time.sleep(sleep_time)
                
    #close the file
    f.close()
    print("Completed")
    return

def main():
    start_time = time.time()
    general_app_store_crawl(links_file, sleep)
    print (time.time() - start_time)
    return

if __name__ == '__main__':
    main()
                

Scraping genre 0.
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=*&page=1#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=*&page=2#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=*&page=3#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=*&page=4#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=*&page=5#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=*&page=6#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=*&page=7#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=*&page=8#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=*&page=9#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=*&page=10#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=*&page=11#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=*&page=12#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8

https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=C&page=23#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=C&page=24#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=C&page=25#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=D&page=1#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=D&page=2#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=D&page=3#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=D&page=4#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=D&page=5#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=D&page=6#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=D&page=7#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=D&page=8#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=D&page=9#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=D&page=10#

https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=K&page=9#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=K&page=10#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=L&page=1#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=L&page=2#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=L&page=3#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=L&page=4#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=L&page=5#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=L&page=6#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=L&page=7#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=L&page=8#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=L&page=9#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=L&page=10#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=L&page=11#p

https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=S&page=10#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=S&page=11#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=S&page=12#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=S&page=13#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=S&page=14#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=S&page=15#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=S&page=16#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=S&page=17#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=S&page=18#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=S&page=19#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=S&page=20#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=S&page=21#page
https://itunes.apple.com/tw/genre/ios-books/id6018?mt=8&letter=S

https://itunes.apple.com/tw/genre/ios-business/id6000?mt=8&letter=A&page=13#page
https://itunes.apple.com/tw/genre/ios-business/id6000?mt=8&letter=A&page=14#page
https://itunes.apple.com/tw/genre/ios-business/id6000?mt=8&letter=A&page=15#page
https://itunes.apple.com/tw/genre/ios-business/id6000?mt=8&letter=A&page=16#page
https://itunes.apple.com/tw/genre/ios-business/id6000?mt=8&letter=A&page=17#page
https://itunes.apple.com/tw/genre/ios-business/id6000?mt=8&letter=A&page=18#page
https://itunes.apple.com/tw/genre/ios-business/id6000?mt=8&letter=A&page=19#page
https://itunes.apple.com/tw/genre/ios-business/id6000?mt=8&letter=A&page=20#page
https://itunes.apple.com/tw/genre/ios-business/id6000?mt=8&letter=A&page=21#page
https://itunes.apple.com/tw/genre/ios-business/id6000?mt=8&letter=A&page=22#page
https://itunes.apple.com/tw/genre/ios-business/id6000?mt=8&letter=A&page=23#page
https://itunes.apple.com/tw/genre/ios-business/id6000?mt=8&letter=A&page=24#page
https://itunes.apple.com/tw/

...

https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=T&page=75#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=T&page=76#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=U&page=1#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=U&page=2#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=U&page=3#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=U&page=4#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=U&page=5#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=U&page=6#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=U&page=7#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=U&page=8#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=U&page=9#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=U&page=10#page
https://itunes.apple.com/

https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=W&page=48#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=W&page=49#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=X&page=1#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=X&page=2#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=X&page=3#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=X&page=4#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=Y&page=1#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=Y&page=2#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=Y&page=3#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=Y&page=4#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=Y&page=5#page
https://itunes.apple.com/tw/genre/ios-lifestyle/id6012?mt=8&letter=Y&page=6#page
https://itunes.apple.com/t

https://itunes.apple.com/tw/genre/ios-magazines-newspapers/id6021?mt=8&letter=P&page=4#page
https://itunes.apple.com/tw/genre/ios-magazines-newspapers/id6021?mt=8&letter=P&page=5#page
https://itunes.apple.com/tw/genre/ios-magazines-newspapers/id6021?mt=8&letter=P&page=6#page
https://itunes.apple.com/tw/genre/ios-magazines-newspapers/id6021?mt=8&letter=Q
https://itunes.apple.com/tw/genre/ios-magazines-newspapers/id6021?mt=8&letter=R&page=1#page
https://itunes.apple.com/tw/genre/ios-magazines-newspapers/id6021?mt=8&letter=R&page=2#page
https://itunes.apple.com/tw/genre/ios-magazines-newspapers/id6021?mt=8&letter=R&page=3#page
https://itunes.apple.com/tw/genre/ios-magazines-newspapers/id6021?mt=8&letter=R&page=4#page
https://itunes.apple.com/tw/genre/ios-magazines-newspapers/id6021?mt=8&letter=S&page=1#page
https://itunes.apple.com/tw/genre/ios-magazines-newspapers/id6021?mt=8&letter=S&page=2#page
https://itunes.apple.com/tw/genre/ios-magazines-newspapers/id6021?mt=8&letter=S&page=3#page


https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=D&page=6#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=D&page=7#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=D&page=8#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=D&page=9#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=D&page=10#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=D&page=11#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=D&page=12#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=D&page=13#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=D&page=14#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=D&page=15#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=D&page=16#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=D&page=17#page
https://itunes.apple.com/tw/genre/ios-medica

https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=M&page=15#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=M&page=16#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=M&page=17#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=M&page=18#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=M&page=19#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=M&page=20#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=M&page=21#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=M&page=22#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=M&page=23#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=M&page=24#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=M&page=25#page
https://itunes.apple.com/tw/genre/ios-medical/id6020?mt=8&letter=M&page=26#page
https://itunes.apple.com/tw/genre/ios-me